In [1]:
import numpy as np

In [14]:
def split_sessions(fname, exclude_zero_buys = True):
    tokenized = [[],[]]
    raw = open(fname)
    for line in raw.readlines():
        tokens = line.translate(None, '\n').split(";")
        if exclude_zero_buys:
            if tokens[1] =='':
                pass
            else:
                tokenized[0].append([int(i) for i in tokens[0].split(',')])
                tokenized[1].append([int(i) for i in tokens[1].split(',')])
        else:
            tokenized[0].append([int(i) for i in tokens[0].split(',')])
            if tokens[1] != '':
                tokenized[1].append([int(i) for i in tokens[1].split(',')])
            else:
                tokenized[1].append([])
    return tokenized[0], tokenized[1]

test_tokensviews, test_tokenspurch = split_sessions('./coursera_sessions_test.txt', exclude_zero_buys=False)
train_tokensviews, train_tokenspurch = split_sessions('./coursera_sessions_train.txt', exclude_zero_buys=False)

In [15]:
print test_tokensviews[:5]
print test_tokenspurch[:5]

[[6, 7, 8], [13, 14, 15], [22, 23], [28, 29, 30, 31, 32, 33], [40, 41]]
[[], [], [], [], []]


In [16]:
from collections import Counter
def freqs(views, purch):
    freq_dict_views = Counter()
    freq_dict_purch = Counter()
    for i,j in zip(views, purch):
        for id in i:
            freq_dict_views[id] += 1
        for id in j:
            freq_dict_purch[id] += 1
    return freq_dict_views, freq_dict_purch

In [17]:
test_freq_view, test_freq_purch = freqs(test_tokensviews, test_tokenspurch)
train_freq_view, train_freq_purch = freqs(train_tokensviews, train_tokenspurch)

In [18]:
train_tokenspurch

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [67, 60, 63],
 [],
 [],
 [86],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [199],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [303],
 [],
 [],
 [352],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [519],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [603, 604, 602, 599, 605, 606, 600],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [690, 688],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [851],
 [],
 [],
 [],
 [879],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1118],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1545],
 [],
 [],
 [],
 

In [19]:
test_view_ind = sorted(test_freq_view, key=test_freq_view.get, reverse=True)
test_purch_ind = sorted(test_freq_purch, key=test_freq_purch.get, reverse=True)
train_view_ind = sorted(train_freq_view, key=train_freq_view.get, reverse=True)
train_purch_ind = sorted(train_freq_purch, key=train_freq_purch.get, reverse=True)

In [20]:
import itertools
def precision(recomendation, purch, fixlen):
    cnt = 0
    for i,j in itertools.product(recomendation, purch):
        if i == j:
             cnt += 1
    return float(cnt)/fixlen

def recall(recomendation, purch):
    cnt = 0
    for i,j in itertools.product(recomendation, purch):
        if i == j:
             cnt += 1
    return float(cnt)/len(purch)

In [21]:
test_tokensviews, test_tokenspurch = split_sessions('./coursera_sessions_test.txt', exclude_zero_buys=True)
train_tokensviews, train_tokenspurch = split_sessions('./coursera_sessions_train.txt', exclude_zero_buys=True)

In [22]:
from more_itertools import unique_everseen

In [23]:
def init_cmp(base_list, base_dict):
    def cmp(x,y):
        if (base_dict[x] > base_dict[y]):
            return -1
        elif base_dict[x] < base_dict[y]:
            return 1
        else:
            if base_list.index(x) < base_list.index(y):
                return -1
            else:
                return 1
    return cmp
    
num_rec = [1,5]
for number_of_recomendations in num_rec:
    for rec_ind, (purch, view) in [[train_view_ind, (test_tokenspurch, test_tokensviews)],\
                                  [train_view_ind, (train_tokenspurch, train_tokensviews)]]:
        preciosions = []
        recalls = []
        refrecommendation = rec_ind[:number_of_recomendations]
        print refrecommendation
        for session_purch, session_view in zip(purch, view):
            cmp_func = init_cmp(session_view, train_freq_purch)
            session_view_s = list(unique_everseen(sorted(session_view, cmp=cmp_func)))
            recommendation = session_view_s[:min(number_of_recomendations,len(session_view_s))]
            #print recommendation, session_purch
            preciosions.append(precision(purch = session_purch, recomendation=recommendation, fixlen=number_of_recomendations))
            recalls.append(recall(purch = session_purch, recomendation=recommendation))
        #print recalls
        print (np.array(recalls).mean(), np.array(preciosions).mean())

[73]
(0.4606201666660298, 0.5276944065484311)
[73]
(0.6884494924267651, 0.8037694013303769)
[73, 158, 204, 262, 162]
(0.8201874337490196, 0.21009549795361532)
[73, 158, 204, 262, 162]
(0.9263073024228791, 0.25254988913525506)


In [24]:
num_rec = [1,5]
for number_of_recomendations in num_rec:
    for rec_ind, (purch, view) in [[train_purch_ind, (test_tokenspurch, test_tokensviews)],\
                                  [train_purch_ind, (train_tokenspurch, train_tokensviews)]]:
        preciosions = []
        recalls = []
        refrecommendation = rec_ind[:number_of_recomendations]
        print refrecommendation
        for session_purch, session_view in zip(purch, view):
            recommendation = rec_ind[:min(number_of_recomendations,len(session_view))]
            preciosions.append(precision(purch = session_purch, recomendation=recommendation, fixlen=number_of_recomendations))
            recalls.append(recall(purch = session_purch, recomendation=recommendation))
        print (np.array(preciosions).mean(), np.array(recalls).mean())

[158]
(0.001091405184174625, 0.0008731241473396999)
[158]
(0.003880266075388027, 0.0027208056171470806)
[158, 204, 73, 3324, 977]
(0.0014188267394270122, 0.004181579862504347)
[158, 204, 73, 3324, 977]
(0.002660753880266076, 0.007612272551296942)


In [25]:
f = open('first.txt', 'w')
f.write('0.44 0.51 0.82 0.21')
f.close()

In [26]:
f = open('second.txt', 'w')
f.write('0.42 0.48 0.80 0.20')
f.close()

In [27]:
f = open('third.txt', 'w')
f.write('0.69 0.80 0.93 0.25')
f.close()

In [28]:
f = open('fourth.txt', 'w')
f.write('0.46 0.53 0.82 0.21')
f.close()